In [317]:
from aocd.models import Puzzle

puzzle = Puzzle(year=2022, day=21)

def parses(input):
    values = {}
    for line in input.strip().split('\n'):
        k, v = line.split(':')
        v = v.split()
        values[k] = int(v[0]) if len(v) == 1 else tuple(v)
    return values
data = parses(puzzle.input_data)

In [301]:
sample = parses(
    """root: pppw + sjmn
dbpl: 5
cczh: sllz + lgvd
zczc: 2
ptdq: humn - dvpt
dvpt: 3
lfqf: 4
humn: 5
ljgn: 2
sjmn: drzm * dbpl
sllz: 4
pppw: cczh / lfqf
lgvd: ljgn * ptdq
drzm: hmdt - zczc
hmdt: 32
"""
)

In [302]:
import functools
import operator
import z3


In [303]:
def eval_expression(data):
    ops = {'+': operator.add, '-': operator.sub, '*': operator.mul, '/': operator.truediv, '=': operator.eq}
    @functools.lru_cache(maxsize=None)
    def memoize(monkey):
        v = data[monkey]
        if not isinstance(v, tuple):
            return v
        a, op, b = v
        return ops[op](memoize(a), memoize(b))    
    return memoize('root')

In [275]:
def solve_a(data):
    return eval_expression(data)

In [276]:
solve_a(sample)

152.0

In [278]:
def solve_b_z3(data):
    x = z3.Real('x')
    data = data.copy()
    data['humn'] = x
    a, _, b = data['root']
    data['root'] = a, '=', b
    
    solver = z3.Solver()
    solver.add(eval_expression(data))
    if solver.check():
        return solver.model()[x].as_long()

In [282]:
solve_b_z3(sample), solve_b_z3(data)

(301, 3916491093817)

In [289]:
from sympy.solvers import solve
from sympy.solvers import 
from sympy import Symbol

In [298]:
def solve_b_sympy(data):
    x = Symbol('x')
    data = data.copy()
    data['humn'] = x
    a, _, b = data['root']
    data['root'] = a, '-', b
    eq = eval_expression(data)
    return round(solve(eq, x)[0])

In [299]:
solve_b_sympy(sample), solve_b_sympy(data)

(301, 3916491093817)

In [280]:
def solve_b_manual(data):
    ops = {'+': operator.add, '-': operator.sub, '*': operator.mul, '/': operator.truediv}
    simplified = {}
    to_simplify = []

    @functools.lru_cache(maxsize=None)
    def can_simplify(monkey):
        if monkey == 'humn':
            return False          

        v = data[monkey]
        if isinstance(v, int):
            simplified[monkey] = v
            return True
        a, op, b = v
        sa, sb = can_simplify(a), can_simplify(b)
        if sa and sb: # ops over scalars
            simplified[monkey] = ops[op](simplified[a], simplified[b])
            return True
        else:
            to_simplify.append(monkey)
            return False

    can_simplify('root')

    while to_simplify:
        monkey = to_simplify.pop()
        a, op, b = data[monkey]
        a_ops = {'+': operator.sub, '-': operator.add, '*': operator.truediv, '/': operator.mul}
        b_ops = {'+': operator.sub, '-': lambda x,y: y-x, '*': operator.truediv, '/': operator.truediv}
        if monkey == 'root':
            if a in simplified:
                simplified[b] = simplified[a]
            else:
                simplified[a] = simplified[b]
        else:
            if a in simplified:
                simplified[b] = b_ops[op](simplified[monkey], simplified[a])
            else:
                simplified[a] = a_ops[op](simplified[monkey], simplified[b])
    return int(simplified['humn'])

In [288]:
solve_b_manual(sample), solve_b_manual(data)

(301, 3916491093817)

In [272]:
solve_b_sympy(sample)

301

In [329]:
def eval_expression(data):
    ops = {'+': operator.add, '-': operator.sub, '*': operator.mul, '/': operator.truediv, '=': operator.eq, '_': lambda *x: x}
    @functools.lru_cache(maxsize=None)
    def memoize(monkey):
        v = data[monkey]
        if not isinstance(v, tuple):
            return v
        a, op, b = v
        return ops[op](memoize(a), memoize(b))    
    return memoize('root')

In [330]:
sample['root']

('pppw', '+', 'sjmn')

In [331]:
data['root']

('lccz', '+', 'pttp')

In [339]:
data = data.copy()
data['humn'] = 1j
a, _, b = data['root']
data['root'] = a, '_', b
eq = eval_expression(data)
a, b = eq
if not isinstance(a, complex):
    a, b = b, a
return int((b - a.real)/a.imag)

3916491093817

In [311]:
a, b = eq.real, eq.imag

In [313]:
eq

(149.5+0.5j)

In [312]:
a/b

299.0

In [262]:
solve_b_z3(sample)

(4 + 2*(x - 3))/4 == 150


301

In [108]:
def solve_a(data):
    ops = {'+': operator.add, '-': operator.sub, '*': operator.mul, '/': operator.floordiv}
    @functools.lru_cache(maxsize=None)
    def memoize(monkey):
        v = data[monkey]
        if isinstance(v, int):
            return v
        a, op, b = v
        return ops[op](memoize(a), memoize(b))    
    return memoize('root')

In [109]:
solve_a(sample)

152

In [110]:
solve_a(data)

72664227897438

In [182]:
def solve_b(data):
    ops = {'+': operator.add, '-': operator.sub, '*': operator.mul, '/': operator.truediv}
    x = z3.Real('x')
    
    @functools.lru_cache(maxsize=None)
    def memoize(monkey):
        if monkey == 'humn':
            return x
        v = data[monkey]
        if isinstance(v, int):
            return v
        a, op, b = v
        if monkey == 'root':
            return memoize(a) == memoize(b)
        return ops[op](memoize(a), memoize(b))    
    
    solver = z3.Solver()
    solver.add(memoize('root'))
    if solver.check():
        return solver.model()[x].as_long()

In [183]:
solve_b(data)

3916491093817

In [134]:
data

{'gdnq': 2,
 'snmj': ['lncs', '*', 'jrrn'],
 'rcfv': 4,
 'mcpm': 4,
 'fvdp': 2,
 'hldt': ['lcvn', '*', 'bbbb'],
 'jzvz': ['hhgs', '+', 'dpzm'],
 'vtww': 3,
 'vjqg': 2,
 'jtrj': ['hsmh', '*', 'qfsv'],
 'gdbg': 10,
 'sqdv': 2,
 'jrfv': ['swzv', '*', 'gnsj'],
 'vvwj': 9,
 'vjdz': 2,
 'lzbv': ['rsgt', '*', 'jmzq'],
 'plng': ['ttjj', '/', 'lqhz'],
 'szts': 2,
 'zzpm': 8,
 'jnjm': ['crlq', '+', 'ldrh'],
 'qvrl': 2,
 'ltrc': 3,
 'rhsw': 1,
 'gbsj': ['bhpj', '-', 'qmqt'],
 'qsdd': 2,
 'sfwh': ['sqbv', '-', 'ftmm'],
 'pmvt': ['ltdp', '/', 'vlwl'],
 'nzzn': ['grmr', '+', 'hhdh'],
 'hjhv': ['cqzd', '+', 'fblr'],
 'plbh': ['fmnv', '/', 'gqcz'],
 'dtzr': ['nmnj', '+', 'wqfn'],
 'hmsn': 3,
 'fnsh': ['sdgj', '*', 'cqlh'],
 'jtsw': 2,
 'bvsg': ['tqhh', '*', 'lsvq'],
 'vbsf': 5,
 'hsqj': 1,
 'hmwj': ['hhtq', '*', 'nhzd'],
 'lbdf': ['pjfm', '*', 'cghz'],
 'fcbn': ['rhmt', '*', 'wgff'],
 'frhm': ['swqt', '+', 'tztc'],
 'qlgt': ['pcqt', '-', 'hwvf'],
 'dhdf': ['jgvq', '/', 'sbls'],
 'rhfw': 2,
 'qnfh': ['

In [130]:


def solve_b(data):
    ops = {'+': operator.add, '-': operator.sub, '*': operator.mul, '/': operator.truediv}
    x = Symbol('x')
    @functools.lru_cache(maxsize=None)
    def memoize(monkey):
        if monkey == 'humn':
            return x
        v = data[monkey]
        if isinstance(v, int):
            return v
        a, op, b = v
        if monkey == 'root':
            return memoize(a) - memoize(b)
        return ops[op](memoize(a), memoize(b))    
    
    print(memoize('root'))
    return int(solve(memoize('root'), x)[0])


In [131]:
solve_b(sample)

x/2 - 301/2


301

In [132]:
solve_b(data)

28717066548639.4 - 14848*x/2025


3916491093816

In [134]:
data

{'gdnq': 2,
 'snmj': ['lncs', '*', 'jrrn'],
 'rcfv': 4,
 'mcpm': 4,
 'fvdp': 2,
 'hldt': ['lcvn', '*', 'bbbb'],
 'jzvz': ['hhgs', '+', 'dpzm'],
 'vtww': 3,
 'vjqg': 2,
 'jtrj': ['hsmh', '*', 'qfsv'],
 'gdbg': 10,
 'sqdv': 2,
 'jrfv': ['swzv', '*', 'gnsj'],
 'vvwj': 9,
 'vjdz': 2,
 'lzbv': ['rsgt', '*', 'jmzq'],
 'plng': ['ttjj', '/', 'lqhz'],
 'szts': 2,
 'zzpm': 8,
 'jnjm': ['crlq', '+', 'ldrh'],
 'qvrl': 2,
 'ltrc': 3,
 'rhsw': 1,
 'gbsj': ['bhpj', '-', 'qmqt'],
 'qsdd': 2,
 'sfwh': ['sqbv', '-', 'ftmm'],
 'pmvt': ['ltdp', '/', 'vlwl'],
 'nzzn': ['grmr', '+', 'hhdh'],
 'hjhv': ['cqzd', '+', 'fblr'],
 'plbh': ['fmnv', '/', 'gqcz'],
 'dtzr': ['nmnj', '+', 'wqfn'],
 'hmsn': 3,
 'fnsh': ['sdgj', '*', 'cqlh'],
 'jtsw': 2,
 'bvsg': ['tqhh', '*', 'lsvq'],
 'vbsf': 5,
 'hsqj': 1,
 'hmwj': ['hhtq', '*', 'nhzd'],
 'lbdf': ['pjfm', '*', 'cghz'],
 'fcbn': ['rhmt', '*', 'wgff'],
 'frhm': ['swqt', '+', 'tztc'],
 'qlgt': ['pcqt', '-', 'hwvf'],
 'dhdf': ['jgvq', '/', 'sbls'],
 'rhfw': 2,
 'qnfh': ['

In [171]:
def solve_b_manual(data):
    ops = {'+': operator.add, '-': operator.sub, '*': operator.mul, '/': operator.truediv}
    simplified = {}
    to_simplify = []

    @functools.lru_cache(maxsize=None)
    def can_simplify(monkey):
        if monkey == 'humn':
            return False          

        v = data[monkey]
        if isinstance(v, int):
            simplified[monkey] = v
            return True
        a, op, b = v
        sa, sb = can_simplify(a), can_simplify(b)
        if sa and sb: # ops over scalars
            simplified[monkey] = ops[op](simplified[a], simplified[b])
            return True
        else:
            to_simplify.append(monkey)
            return False

    can_simplify('root')

    while to_simplify:
        monkey = to_simplify.pop()
        a, op, b = data[monkey]
        a_ops = {'+': operator.sub, '-': operator.add, '*': operator.truediv, '/': operator.mul}
        b_ops = {'+': operator.sub, '-': lambda x,y: y-x, '*': operator.truediv, '/': operator.truediv}
        if monkey == 'root':
            if a in simplified:
                simplified[b] = simplified[a]
            else:
                simplified[a] = simplified[b]
        else:
            if a in simplified:
                simplified[b] = b_ops[op](simplified[monkey], simplified[a])
            else:
                simplified[a] = a_ops[op](simplified[monkey], simplified[b])
    return int(simplified['humn'])

In [174]:
solve_b_manual(data)

3916491093817

In [173]:
to_simplify

[]

In [164]:
to_simplify

[]

In [165]:
int(simplified['humn'])

301

In [ ]:
def solve_b(data):
    ops = {'+': operator.add, '-': operator.sub, '*': operator.mul, '/': operator.truediv}
    simplified = {}
    to_simplify = []
    
    @functools.lru_cache(maxsize=None)
    def can_simplify(monkey):
        if monkey == 'humn':
            to_simplify.append(monkey)
            return False          
        # scalars
        v = data[monkey]
        if isinstance(v, int):
            simplified[monkey] = v
            return True
        a, op, b = v
        a, b = memoize(a), memoize(b)
        if a and b: # ops over scalars
            simplified[monkey] = ops[op](simplified[a], simplified[b])
            return True
        else:
            to_simplify.append(monkey)
            return False
        
    can_simplify(root)